In [2]:
#coding=utf-8
import numpy as np
import pandas as pd
from scipy.stats import mode
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from matplotlib import  pyplot
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor

llb


In [3]:
train = pd.read_table("../input/contest_basic_train.tsv")
test = pd.read_table("../input/contest_basic_test.tsv")
print(train.head())
print(train.shape)
# print(test.head())    # (30000, 11)
print(test.shape)  #  (10000, 10)
fulldata = pd.concat([train.iloc[:,0:-1],test],axis = 0,ignore_index = True)

fulldata.index = fulldata['REPORT_ID']
del fulldata['REPORT_ID']
print(fulldata.head())
print(fulldata.shape)  #  (40000, 9),ID 用来做  index
# print(fulldata. MARRY_STATUS)
print(fulldata.SALARY.unique())

   REPORT_ID             ID_CARD  LOAN_DATE   AGENT IS_LOCAL  WORK_PROVINCE  \
0       8787  320382**********13  2017/2/15     APP      本地籍       320000.0   
1       9410  120223**********8X  2017/3/13  wechat      本地籍       120000.0   
2      22999  330226**********90  2017/2/27     APP      本地籍            NaN   
3      24820  230105**********20  2017/2/24     APP      本地籍       230000.0   
4      25708  340222**********29  2017/1/16     NaN      本地籍            NaN   

  EDU_LEVEL MARRY_STATUS  SALARY  HAS_FUND  Y  
0        本科           已婚     3.0       0.0  0  
1       NaN           已婚     6.0       1.0  0  
2     专科及以下           已婚     4.0       0.0  0  
3     专科及以下           已婚     2.0       0.0  1  
4        专科           未婚     NaN       1.0  0  
(30000, 11)
(10000, 10)
                      ID_CARD  LOAN_DATE   AGENT IS_LOCAL  WORK_PROVINCE  \
REPORT_ID                                                                  
8787       320382**********13  2017/2/15     APP      本地籍    

In [4]:
print(fulldata.shape)
null_sum = fulldata.isnull().sum()
# print(null_sum)
null_sum = null_sum[null_sum !=0]  #把没有缺失值的剔除掉，对于数据集维度大的数据有效
null_sum_df = pd.DataFrame(null_sum,columns=['num'])  # 对缺失值这一列进行设为DataFrame
null_sum_df['ratio'] = null_sum_df['num'] / fulldata.shape[0]
# print(null_sum_df)
null_sum_df.sort_values(by='ratio',ascending=False,inplace=True)
print(null_sum_df)
# print(fulldata.head())

(40000, 9)
                 num     ratio
SALARY         26666  0.666650
AGENT          26174  0.654350
EDU_LEVEL       5593  0.139825
WORK_PROVINCE   2925  0.073125
HAS_FUND        1079  0.026975
IS_LOCAL         276  0.006900
MARRY_STATUS       1  0.000025


In [5]:
#按行统计 测试集最大的缺失值个数是否小于 训练集的，是，删除
train_record_nan = train.isnull().sum(axis = 1).sort_values(ascending = False)  #按行统计缺失值
print(train_record_nan.unique())  # [4 3 2 1 0]
test_record_nan = test.isnull().sum(axis= 1).sort_values(ascending = False)
print(test_record_nan.unique())  # [5 4 3 2 1 0]
#说明训练集和测试集缺失值差异不大，不用删除行

#然后，每一行的缺失值个数作为 一个新的特征，连续值特征
# print(fulldata.isnull().sum(axis=1))
fulldata['null_num'] = fulldata.isnull().sum(axis = 1)

# 薪水缺失值较大，用-1填充，客户渠道用-1 填充 也许是没有统计到的。其余用众数填充
#字符先转换成数字
fulldata['AGENT'].ix[fulldata['AGENT'].isnull()] = -1
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# print(11,fulldata.MARRY_STATUS.unique())

fulldata['AGENT'] = fulldata['AGENT'].astype('str')
c= le.fit_transform(fulldata['AGENT']) #文字不行
fulldata['AGENT'] = c

fulldata['IS_LOCAL'] = fulldata['IS_LOCAL'].astype('str')
c= le.fit_transform(fulldata['IS_LOCAL']) #文字不行
fulldata['IS_LOCAL'] = c

fulldata['EDU_LEVEL'] = fulldata['EDU_LEVEL'].astype('str')
c= le.fit_transform(fulldata['EDU_LEVEL']) #文字不行
fulldata['EDU_LEVEL'] = c

fulldata['MARRY_STATUS'] = fulldata['MARRY_STATUS'].astype('str')
c= le.fit_transform(fulldata['MARRY_STATUS']) #文字不行
fulldata['MARRY_STATUS'] = c

# # fulldata['AGENT'] = c.labels
# print(fulldata.head())


[4 3 2 1 0]
[5 4 3 2 1 0]


c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [6]:
label = fulldata.columns
for i in range(len(label)):
    if fulldata[label[i]].isnull().any() and label[i] != 'SALARY':
        fulldata[label[i]].fillna(fulldata[label[i]].median(),inplace = True)
print(fulldata.iloc[0:5,:])

                      ID_CARD  LOAN_DATE  AGENT  IS_LOCAL  WORK_PROVINCE  \
REPORT_ID                                                                  
8787       320382**********13  2017/2/15      1         1       320000.0   
9410       120223**********8X  2017/3/13     11         1       120000.0   
22999      330226**********90  2017/2/27      1         1       330600.0   
24820      230105**********20  2017/2/24      1         1       230000.0   
25708      340222**********29  2017/1/16      0         1       330600.0   

           EDU_LEVEL  MARRY_STATUS  SALARY  HAS_FUND  null_num  
REPORT_ID                                                       
8787               6             3     3.0       0.0         0  
9410               0             3     6.0       1.0         1  
22999              2             3     4.0       0.0         1  
24820              2             3     2.0       0.0         0  
25708              1             4     NaN       1.0         3  


#  薪水缺失值较多（60%以上），先使用随机森林填充

In [7]:
xgb = XGBClassifier(
    learning_rate = 0.1,
    n_estimators = 80,
    max_depth = 4,
    min_child_weight = 4,
    colsample_bytree = 0.7,
    subsample = 0.75,
    seed = 3
)
mm = pd.concat([fulldata.AGENT,fulldata.IS_LOCAL,fulldata.WORK_PROVINCE,fulldata.EDU_LEVEL,fulldata.MARRY_STATUS,
                    fulldata.HAS_FUND,fulldata.null_num,fulldata.SALARY],axis = 1)
nn = train.Y
mm_notnull = mm.ix[mm.SALARY.notnull()]
mm_isnull = mm.ix[mm.SALARY.isnull()]

# trainx = trainxxx.loc[train.SALARY.notnull()] # 这样写严重错误，因为这个数据集中根本就没有这一列
xgb.fit(mm_notnull.iloc[:,0:-1],mm_notnull.iloc[:,-1])
fulldata.SALARY.ix[fulldata.SALARY.isnull()] = xgb.predict(mm_isnull.iloc[:,0:-1])

c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  del sys.path[0]
c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_inde

In [8]:
print(fulldata.head())

                      ID_CARD  LOAN_DATE  AGENT  IS_LOCAL  WORK_PROVINCE  \
REPORT_ID                                                                  
8787       320382**********13  2017/2/15      1         1       320000.0   
9410       120223**********8X  2017/3/13     11         1       120000.0   
22999      330226**********90  2017/2/27      1         1       330600.0   
24820      230105**********20  2017/2/24      1         1       230000.0   
25708      340222**********29  2017/1/16      0         1       330600.0   

           EDU_LEVEL  MARRY_STATUS  SALARY  HAS_FUND  null_num  
REPORT_ID                                                       
8787               6             3     3.0       0.0         0  
9410               0             3     6.0       1.0         1  
22999              2             3     4.0       0.0         1  
24820              2             3     2.0       0.0         0  
25708              1             4     2.0       1.0         3  


# 下面除了身份证，时间，工作城市外其余数据one-hot 编码 保存起来

In [229]:
#后面再编码，先处理特殊类别
AGENT = pd.get_dummies(fulldata.AGENT,prefix = 'AGENT')
IS_LOCAL = pd.get_dummies(fulldata.IS_LOCAL,prefix = 'IS_LOCAL')
EDU_LEVEL = pd.get_dummies(fulldata.EDU_LEVEL,prefix = 'EDU_LEVEL')
MARRY_STATUS = pd.get_dummies(fulldata.MARRY_STATUS,prefix = 'MARRY_STATUS')
HAS_FUND = pd.get_dummies(fulldata.HAS_FUND,prefix = 'HAS_FUND')
ff = pd.concat([AGENT,IS_LOCAL,EDU_LEVEL,MARRY_STATUS,HAS_FUND,fulldata.SALARY,fulldata.null_num],axis = 1)
# ff.to_csv("../basic-input/除身份证时间工作城市外所有数据.csv",index = False)
print(ff.shape)

(40000, 45)


In [104]:
print(fulldata['WORK_PROVINCE'].unique().shape) # 为城市编码 ，后面引入经纬度

(323,)


#  下面对身份证号码进行处理：省，区，男性还是女性等

#  身份证号码很有探索的地方：
1.前两位是省份，第3、4位数字表示：所在城市的代码； 第5、6位数字表示：所在区县的代码； 第17位数字表示性别：奇数表示男性，偶数表示女性。总共18位。
猜想：省份有关系，城市有关系（1,2线城市），区还是县也有关系（市里面的人应该更有合约精神，市分一线城市和二线城市也是同样的道理），男性和女性应该也有关系。

猜想：客户渠道应该也有影响，如果是朋友推荐是不是更不可靠一点（全是贷款朋友），第三方机构来的为什么不在原平台？

In [16]:
IC = fulldata["ID_CARD"]
print(IC.iloc[0])
province = []  #省
city = []      #市
district = []  #区县
sex = []
# 首先前两位是省份，先提取，用xgb训练后 得到后保留
kk = fulldata["ID_CARD"].astype('str')
for i in range(fulldata.shape[0]):
    province.append(kk.iloc[i][0:2])
    city.append(kk.iloc[i][2:4])
    district.append(kk.iloc[i][4:6])
    sex.append(int(kk.iloc[i][16])%2) #男性 或 女性
    
province = pd.DataFrame(province,columns = ['province'])
city = pd.DataFrame(city,columns = ['city'])
district = pd.DataFrame(district,columns = ['district'])
sex = pd.DataFrame(sex,columns = ['sex'])
# print(kk[0:10])
# print(2,province.iloc[0:5])
# print(3,city.head())
# print(4,district.head())
# print(5,sex.head())
fulldata.index = sex.index
IDcard = pd.concat([city,district, fulldata.WORK_PROVINCE],axis = 1)
# print(IDcard)
IDcard.index = fulldata.index
# print(IDcard)
IDcard.to_csv("../basic-input/IDcard.csv",index = False)

320382**********13


#  省和市县按照PDF先one-hot编码，得到29个重要的省市区县，经纬度后续引入

In [221]:
# print(1,province['province'].unique().shape)  #中国总共有 34 个省，直辖市，自治区  
# print(1,province['province'].unique())
# # print(sex)
# print(2,city['city'].unique())
# print(2,city['city'].unique().shape)
# print(3,district['district'].unique())
# print(3,district['district'].unique().shape)
# print(train.Y.unique())
province = pd.get_dummies(province,prefix='province')
city = pd.get_dummies(city,prefix='city')
district = pd.get_dummies(district,prefix='district')
trainx = pd.concat([province,city,district],axis=1)
trainy = train.Y
print("trainx.shape",trainx.shape)
print("trainy",trainy.shape)
xgb = XGBClassifier()
xgb.fit(trainx.iloc[0:30000,:],trainy)
# print(trainx.head())
importance = xgb.feature_importances_


trainx.shape (40000, 125)
trainy (30000,)


In [222]:
# sns.distplot(importance)
# importance = pd.Series(importance)
# # print(importance.sort_values(ascending = False))
label = trainx.columns
number = 0
for i in range(len(label)):
    if importance[i] < 0.01:
        number += 1
        del trainx[label[i]]
print(number)
# print(trainx.head())

96


In [237]:
# print(ff.head())
# print(trainx.head())

In [234]:
ff.index = trainx.index
full = pd.concat([ff,trainx,sex],axis = 1)
full.to_csv("../basic-input/basic-fulldata-Nodate-经纬度.csv",index = False)
# trainx.to_csv("../basic-input/省市区县(后缀为身份证号对应的地方编号).csv",index = False)

RangeIndex(start=0, stop=40000, step=1)
